In [1]:
# Import dependencies

import os
import csv
import pandas as pd

#### Clean economic data in preparation for machine learning model

In [2]:
# Read in economic data file
econ_df = pd.read_csv("../CSV_Outputs/simplified_economic.csv",)

# Display sample data
econ_df.head(5)

,State,Economic Indicator,Year,Value
0,Alabama,Gross domestic product (GDP),2008,174665
1,Alabama,Personal income,2008,157780
2,Alabama,Total employment (number of jobs),2008,2582600
3,Alaska,Gross domestic product (GDP),2008,55246
4,Alaska,Personal income,2008,32860


In [3]:
# Pivot the DataFrame
econ_df = econ_df.pivot_table(index=['State', 'Year'], columns='Economic Indicator', values='Value', aggfunc='first').reset_index()

econ_df.head()

Economic Indicator,State,Year,Gross domestic product (GDP),Personal income,Total employment (number of jobs)
0,Alabama,2008,174665,157780,2582600
1,Alabama,2012,188280,172101,2503656
2,Alabama,2016,208824,190871,2619761
3,Alabama,2020,235118,230872,2671005
4,Alaska,2008,55246,32860,443602


In [4]:
# Flatten the column names (optional)
econ_df.columns.name = None
econ_df = econ_df.rename_axis(None, axis=1)

econ_df.head()

,State,Year,Gross domestic product (GDP),Personal income,Total employment (number of jobs)
0,Alabama,2008,174665,157780,2582600
1,Alabama,2012,188280,172101,2503656
2,Alabama,2016,208824,190871,2619761
3,Alabama,2020,235118,230872,2671005
4,Alaska,2008,55246,32860,443602


In [5]:
# Read in election data CSV file
elec_df = pd.read_csv("../CSV_Outputs/merged_election.csv",)

# Display sample data
elec_df.head(5)

,state,candidate,total_votes,party,Election Year
0,Alabama,"McCain, John",1266546,Republican,2008
1,Alabama,"Obama, Barack",813479,Democrat,2008
2,Alaska,"McCain, John",193841,Republican,2008
3,Alaska,"Obama, Barack",123594,Democrat,2008
4,Arizona,"McCain, John",1230111,Republican,2008


In [6]:
# Prepare data to merge the election and economic dataframes

# Drop the 'Candidate' column from the elec_df DataFrame
elec_df = elec_df.drop(columns=['candidate'])

# Rename columns in elec_df to match those in econ_df
elec_df = elec_df.rename(columns={
    'state': 'State',
    'Election Year': 'Year',
    'total_votes': 'Total Votes',
    'party': 'Candidate'
})

elec_df.head()

,State,Total Votes,Candidate,Year
0,Alabama,1266546,Republican,2008
1,Alabama,813479,Democrat,2008
2,Alaska,193841,Republican,2008
3,Alaska,123594,Democrat,2008
4,Arizona,1230111,Republican,2008


In [7]:
# Merge the dataframes on 'State' and 'Year'
merged_df = pd.merge(econ_df, elec_df, on=['State', 'Year'], how='inner')
merged_df.head()

,State,Year,Gross domestic product (GDP),Personal income,Total employment (number of jobs),Total Votes,Candidate
0,Alabama,2008,174665,157780,2582600,1266546,Republican
1,Alabama,2008,174665,157780,2582600,813479,Democrat
2,Alabama,2012,188280,172101,2503656,1255925,Republican
3,Alabama,2012,188280,172101,2503656,795696,Democrat
4,Alabama,2016,208824,190871,2619761,544,Democrat


In [8]:
# Rename columns as required
merged_df = merged_df.rename(columns={
    'Gross domestic product (GDP)': 'GDP',
    'Personal income': 'Personal Income',
    'Total employment (number of jobs)': 'Total Employment'
})

# Reorder columns to match the desired output
merged_df = merged_df[['State', 'Year', 'Candidate', 'Total Votes', 'GDP', 'Personal Income', 'Total Employment']]
merged_df.head()

,State,Year,Candidate,Total Votes,GDP,Personal Income,Total Employment
0,Alabama,2008,Republican,1266546,174665,157780,2582600
1,Alabama,2008,Democrat,813479,174665,157780,2582600
2,Alabama,2012,Republican,1255925,188280,172101,2503656
3,Alabama,2012,Democrat,795696,188280,172101,2503656
4,Alabama,2016,Democrat,544,208824,190871,2619761


#### Append merged_df to include the single Average Sentiment Score column 

In [9]:
# Read in demographic data race file
sent_df = pd.read_csv("../CSV_Outputs/average_sentiment_by_party_all_years.csv",)

# Display sample data
sent_df.head()

,Year,Candidate,Avg_Sentiment
0,2016,Democrat,0.364082
1,2016,Republican,0.403678
2,2020,Democrat,0.001620
3,2020,Republican,-0.002182
4,2024,Republican,0.451690


In [10]:
# Initialize an empty dictionary to store the sentiment scores
sentiment_dict = {}

# Iterate over the rows of the DataFrame
for index, row in sent_df.iterrows():
    # Create a key for the dictionary based on Year and Party
    key = f"{row['Candidate'][:3]}_Sent_{row['Year']}"
    
    # Assign the sentiment score to the dictionary
    sentiment_dict[key] = row['Avg_Sentiment']

# Print the final sentiment_dict to check
print("Final sentiment_dict:", sentiment_dict)

Final sentiment_dict: {'Dem_Sent_2016': 0.3640820575295926, 'Rep_Sent_2016': 0.4036780355948681, 'Dem_Sent_2020': 0.0016201125280288, 'Rep_Sent_2020': -0.0021822172428247, 'Rep_Sent_2024': 0.4516899923638214, 'Dem_Sent_2024': 0.3788153863027722}


In [11]:
# Create a new column in merged_df for Average Sentiment Score
merged_df['Average Sentiment Score'] = None

# Populate the new column based on the sentiment_dict
for index, row in merged_df.iterrows():
    # Create the key for the dictionary based on Year and Party
    key = f"{row['Candidate'][:3]}_Sent_{row['Year']}"
    
    # Assign the sentiment score to the new column if the key exists in the dictionary
    if key in sentiment_dict:
        merged_df.at[index, 'Average Sentiment Score'] = sentiment_dict[key]

# Show the updated DataFrame
merged_df

,State,Year,Candidate,Total Votes,GDP,Personal Income,Total Employment,Average Sentiment Score
0,Alabama,2008,Republican,1266546,174665,157780,2582600,None
1,Alabama,2008,Democrat,813479,174665,157780,2582600,None
2,Alabama,2012,Republican,1255925,188280,172101,2503656,None
3,Alabama,2012,Democrat,795696,188280,172101,2503656,None
4,Alabama,2016,Democrat,544,208824,190871,2619761,0.364082
...,...,...,...,...,...,...,...,...
34072,Wyoming,2020,Democrat,1591,36675,37866,401701,0.00162
34073,Wyoming,2020,Republican,3245,36675,37866,401701,-0.002182
34074,Wyoming,2020,Democrat,651,36675,37866,401701,0.00162
34075,Wyoming,2020,Republican,3107,36675,37866,401701,-0.002182


In [38]:
# Check for leading or trailing spaces in the 'State' column
merged_df['State'].str.strip().unique()  # Returns unique values with spaces removed
merged_df.head()

,State,Year,Candidate,Total Votes,GDP,Personal Income,Total Employment,Average Sentiment Score
0,Alabama,2008,Republican,1266546,174665,157780,2582600,None
1,Alabama,2008,Democrat,813479,174665,157780,2582600,None
2,Alabama,2012,Republican,1255925,188280,172101,2503656,None
3,Alabama,2012,Democrat,795696,188280,172101,2503656,None
4,Alabama,2016,Democrat,544,208824,190871,2619761,0.364082


In [39]:
# Identify rows where the 'State' column has leading or trailing spaces
rows_with_spaces = merged_df[merged_df['State'] != merged_df['State'].str.strip()]

# Display rows with leading or trailing spaces
print(rows_with_spaces[['State']])

Empty DataFrame
Columns: [State]
Index: []


In [40]:
# Remove leading and trailing spaces from the 'State' column
merged_df['State'] = merged_df['State'].str.strip()
merged_df.head()

,State,Year,Candidate,Total Votes,GDP,Personal Income,Total Employment,Average Sentiment Score
0,Alabama,2008,Republican,1266546,174665,157780,2582600,None
1,Alabama,2008,Democrat,813479,174665,157780,2582600,None
2,Alabama,2012,Republican,1255925,188280,172101,2503656,None
3,Alabama,2012,Democrat,795696,188280,172101,2503656,None
4,Alabama,2016,Democrat,544,208824,190871,2619761,0.364082


In [41]:
# Save the merged economic and sentiment CSV file
merged_df.to_csv('../CSV_Outputs/Econ_Sent_Data.csv', index=False)

#### Clean age census data in preparation for machine learning model

In [13]:
# Read in demographic data race file
age_df = pd.read_csv("../CSV_Outputs/age_demo_data.csv",)

# Display sample data
age_df.head(5)

,state,Population_Value,Age Group,Gender,Year
0,Alabama,47,18+,M,2008
1,New Jersey,6,65-74,NaN,2008
2,New Mexico,7,65-74,NaN,2008
3,New York,6,65-74,NaN,2008
4,North Carolina,6,65-74,NaN,2008


In [14]:
# Drop the Population_Value column
age_df.drop(columns=['Population_Value'], inplace=True)
age_df.head()

,state,Age Group,Gender,Year
0,Alabama,18+,M,2008
1,New Jersey,65-74,NaN,2008
2,New Mexico,65-74,NaN,2008
3,New York,65-74,NaN,2008
4,North Carolina,65-74,NaN,2008


In [15]:
# Get unique age groups
unique_age_groups = age_df['Age Group'].unique()

# Create new columns for each age group
for age_group in unique_age_groups:
    age_df[age_group] = (age_df['Age Group'] == age_group).astype(int)

# Drop the original 'Age Group' column if it's no longer needed
age_df = age_df.drop(columns=['Age Group'])

In [16]:
age_df.head()

,state,Gender,Year,18+,65-74,75-84,55-59,60-64,NaN,20-24,25-34,45-54,35-44
0,Alabama,M,2008,1,0,0,0,0,0,0,0,0,0
1,New Jersey,NaN,2008,0,1,0,0,0,0,0,0,0,0
2,New Mexico,NaN,2008,0,1,0,0,0,0,0,0,0,0
3,New York,NaN,2008,0,1,0,0,0,0,0,0,0,0
4,North Carolina,NaN,2008,0,1,0,0,0,0,0,0,0,0


In [17]:
# Drop columns that contain only NaN values
age_df = age_df.dropna(axis=1, how='all')

age_df.head()

,state,Gender,Year,18+,65-74,75-84,55-59,60-64,NaN,20-24,25-34,45-54,35-44
0,Alabama,M,2008,1,0,0,0,0,0,0,0,0,0
1,New Jersey,NaN,2008,0,1,0,0,0,0,0,0,0,0
2,New Mexico,NaN,2008,0,1,0,0,0,0,0,0,0,0
3,New York,NaN,2008,0,1,0,0,0,0,0,0,0,0
4,North Carolina,NaN,2008,0,1,0,0,0,0,0,0,0,0


In [18]:
# Rename columns as required
age_df = age_df.rename(columns={
    'state': 'State',
    'Population_Value': 'Population Value',
    '18+': '18-20'
})

age_df.head()

,State,Gender,Year,18-20,65-74,75-84,55-59,60-64,NaN,20-24,25-34,45-54,35-44
0,Alabama,M,2008,1,0,0,0,0,0,0,0,0,0
1,New Jersey,NaN,2008,0,1,0,0,0,0,0,0,0,0
2,New Mexico,NaN,2008,0,1,0,0,0,0,0,0,0,0
3,New York,NaN,2008,0,1,0,0,0,0,0,0,0,0
4,North Carolina,NaN,2008,0,1,0,0,0,0,0,0,0,0


In [19]:
# Check for duplicate combinations of state and year
duplicates = age_df[age_df.duplicated(subset=['State', 'Year'], keep=False)]

# Display duplicate rows
print(duplicates)

                State Gender  Year  18-20  65-74  75-84  55-59  60-64  NaN  \
0             Alabama      M  2008      1      0      0      0      0    0   
1          New Jersey    NaN  2008      0      1      0      0      0    0   
2          New Mexico    NaN  2008      0      1      0      0      0    0   
3            New York    NaN  2008      0      1      0      0      0    0   
4      North Carolina    NaN  2008      0      1      0      0      0    0   
...               ...    ...   ...    ...    ...    ...    ...    ...  ...   
2539         Kentucky    NaN  2022      0      0      0      0      0    0   
2540        Louisiana    NaN  2022      0      0      0      0      0    0   
2541            Maine    NaN  2022      0      0      0      0      0    0   
2542         Maryland    NaN  2022      0      0      0      0      0    0   
2543            Idaho    NaN  2022      0      0      0      0      0    0   

      20-24  25-34  45-54  35-44  
0         0      0      0   

In [20]:
# Columns to aggregate
columns_to_sum = ["18-20", "20-24", "25-34", "35-44", "45-54", "55-59", "60-64", "65-74", "75-84"]

# Group by 'state' and 'Year', then aggregate the columns
aggregated_df = age_df.groupby(['State', 'Year'], as_index=False).agg({
    **{col: 'sum' for col in columns_to_sum}
})

# Display the aggregated DataFrame
aggregated_df.head()

,State,Year,18-20,20-24,25-34,35-44,45-54,55-59,60-64,65-74,75-84
0,Alabama,2008,2,1,1,1,1,1,1,1,1
1,Alabama,2012,2,1,1,1,1,1,1,1,1
2,Alabama,2016,2,1,1,1,1,1,1,1,1
3,Alabama,2020,0,0,0,0,0,0,0,0,0
4,Alabama,2022,2,1,1,1,1,1,1,1,1


In [22]:
# Create two copies of the DataFrame with the Candidate column
df_republican = aggregated_df.copy()
df_republican['Candidate'] = 'Republican'

df_democrat = aggregated_df.copy()
df_democrat['Candidate'] = 'Democrat'

In [25]:
# Concatenate the two DataFrames
final_df = pd.concat([df_republican, df_democrat], ignore_index=True)
final_df.head()

,State,Year,18-20,20-24,25-34,35-44,45-54,55-59,60-64,65-74,75-84,Candidate
0,Alabama,2008,2,1,1,1,1,1,1,1,1,Republican
1,Alabama,2012,2,1,1,1,1,1,1,1,1,Republican
2,Alabama,2016,2,1,1,1,1,1,1,1,1,Republican
3,Alabama,2020,0,0,0,0,0,0,0,0,0,Republican
4,Alabama,2022,2,1,1,1,1,1,1,1,1,Republican


In [26]:
# Sort the DataFrame by the 'State' column in descending order
final_df_sorted = final_df.sort_values(by='State', ascending=False)
final_df_sorted.head(20)

,State,Year,18-20,20-24,25-34,35-44,45-54,55-59,60-64,65-74,75-84,Candidate
519,Wyoming,2022,2,1,1,1,1,1,1,1,1,Democrat
256,Wyoming,2012,2,1,1,1,1,1,1,1,1,Republican
515,Wyoming,2008,2,1,1,1,1,1,1,1,1,Democrat
259,Wyoming,2022,2,1,1,1,1,1,1,1,1,Republican
258,Wyoming,2020,0,0,0,0,0,0,0,0,0,Republican
257,Wyoming,2016,2,1,1,1,1,1,1,1,1,Republican
255,Wyoming,2008,2,1,1,1,1,1,1,1,1,Republican
516,Wyoming,2012,2,1,1,1,1,1,1,1,1,Democrat
518,Wyoming,2020,0,0,0,0,0,0,0,0,0,Democrat
517,Wyoming,2016,2,1,1,1,1,1,1,1,1,Democrat


In [27]:
# Save the age census info to a CSV file
final_df_sorted.to_csv('../CSV_Outputs/Merged_Age_Data.csv', index=False)

#### Clean race census data in preparation for machine learning model

In [30]:
# Read in demographic data race file
race_df = pd.read_csv("../CSV_Outputs/merged_race_data2.csv",)

# Display sample data
race_df.head()

,state,white,black_or_african_american,american_indian_and_alaska_native,asian,native_hawaiian_and_other_pacific-islander,hispanic_or_latino_any_race,year
0,Alabama,69.5,27.0,0.5,1.0,0.0,2.8,2008
1,Alaska,69.1,3.6,12.7,4.6,0.5,6.2,2008
2,Arizona,80.0,3.7,4.5,2.4,0.1,30.3,2008
3,Arkansas,78.4,16.0,0.5,1.0,0.1,5.3,2008
4,California,62.0,6.2,0.8,12.6,0.4,36.8,2008


In [31]:
# Create two copies of the DataFrame with the Candidate column
race_df_republican = race_df.copy()
race_df_republican['Candidate'] = 'Republican'

race_df_democrat = race_df.copy()
race_df_democrat['Candidate'] = 'Democrat'

In [32]:
# Concatenate the two DataFrames
final_race_df = pd.concat([race_df_republican, race_df_democrat], ignore_index=True)
final_race_df.head()

,state,white,black_or_african_american,american_indian_and_alaska_native,asian,native_hawaiian_and_other_pacific-islander,hispanic_or_latino_any_race,year,Candidate
0,Alabama,69.5,27.0,0.5,1.0,0.0,2.8,2008,Republican
1,Alaska,69.1,3.6,12.7,4.6,0.5,6.2,2008,Republican
2,Arizona,80.0,3.7,4.5,2.4,0.1,30.3,2008,Republican
3,Arkansas,78.4,16.0,0.5,1.0,0.1,5.3,2008,Republican
4,California,62.0,6.2,0.8,12.6,0.4,36.8,2008,Republican


In [35]:
# Sort the DataFrame by the 'State' column in descending order
final_race_df_sorted = final_race_df.sort_values(by='state', ascending=False)
final_race_df_sorted.head(10)

,state,white,black_or_african_american,american_indian_and_alaska_native,asian,native_hawaiian_and_other_pacific-islander,hispanic_or_latino_any_race,year,Candidate
415,Wyoming,84.7,0.9,2.4,0.9,0.1,10.2,2020,Democrat
207,Wyoming,84.7,0.9,2.4,0.9,0.1,10.2,2020,Republican
363,Wyoming,92.0,1.0,2.3,1.0,0.1,9.6,2016,Democrat
51,Wyoming,91.9,1.1,2.0,0.7,0.0,7.7,2008,Republican
311,Wyoming,90.9,1.0,2.1,1.0,0.0,9.4,2012,Democrat
103,Wyoming,90.9,1.0,2.1,1.0,0.0,9.4,2012,Republican
259,Wyoming,91.9,1.1,2.0,0.7,0.0,7.7,2008,Democrat
155,Wyoming,92.0,1.0,2.3,1.0,0.1,9.6,2016,Republican
50,Wisconsin,88.2,6.0,0.9,2.0,0.0,5.1,2008,Republican
414,Wisconsin,80.2,6.6,1.0,3.0,0.0,7.7,2020,Democrat


In [36]:
# Save the age census info to a CSV file
final_race_df_sorted.to_csv('../CSV_Outputs/Merged_Race_Data.csv', index=False)